Byte-Pair Encoding (BPE) was initially developed as an algorithm to compress texts and it is used in GPT, GPT-2, RoBERTa, BART, and DeBERTa

BPE training starts by **computing the unique set of words** used in the corpus (after the normalization and pre-tokenization steps are completed), then building the vocabulary by taking all the symbols used to write those words


In [3]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

1. use the gpt2 tokenizer for the pre-tokenization

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2") # we’re using a GPT2 tokenizer, as pre-tokenization 
                                                  #involves splitting on whitespace and punctuation

/home/qxy699/hugging_face/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.  compute the frequencies of **each word** in the corpus as we do the pre-tokenization

In [5]:
word_freqs = dict()

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        if word not in word_freqs:
            word_freqs[word] = 1
        else:
            word_freqs[word] += 1

word_freqs

{'This': 3,
 'Ġis': 2,
 'Ġthe': 1,
 'ĠHugging': 1,
 'ĠFace': 1,
 'ĠCourse': 1,
 '.': 4,
 'Ġchapter': 1,
 'Ġabout': 1,
 'Ġtokenization': 1,
 'Ġsection': 1,
 'Ġshows': 1,
 'Ġseveral': 1,
 'Ġtokenizer': 1,
 'Ġalgorithms': 1,
 'Hopefully': 1,
 ',': 1,
 'Ġyou': 1,
 'Ġwill': 1,
 'Ġbe': 1,
 'Ġable': 1,
 'Ġto': 1,
 'Ġunderstand': 1,
 'Ġhow': 1,
 'Ġthey': 1,
 'Ġare': 1,
 'Ġtrained': 1,
 'Ġand': 1,
 'Ġgenerate': 1,
 'Ġtokens': 1}

3. compute the base vocabulary, formed by all the characters used in the corpus

In [6]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

alphabet

[',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'Ġ']

add the special tokens used by the model at the beginning of that vocabulary. In the case of GPT-2, the only special token is "<|endoftext|>"

In [9]:
vocab = ["<|endoftext|>"] + alphabet.copy()
vocab

['<|endoftext|>',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'Ġ']

4. split each word into individual characters, to be able to start training

In [8]:
splits = {word: [c for c in word] for word in word_freqs.keys()}
splits

{'This': ['T', 'h', 'i', 's'],
 'Ġis': ['Ġ', 'i', 's'],
 'Ġthe': ['Ġ', 't', 'h', 'e'],
 'ĠHugging': ['Ġ', 'H', 'u', 'g', 'g', 'i', 'n', 'g'],
 'ĠFace': ['Ġ', 'F', 'a', 'c', 'e'],
 'ĠCourse': ['Ġ', 'C', 'o', 'u', 'r', 's', 'e'],
 '.': ['.'],
 'Ġchapter': ['Ġ', 'c', 'h', 'a', 'p', 't', 'e', 'r'],
 'Ġabout': ['Ġ', 'a', 'b', 'o', 'u', 't'],
 'Ġtokenization': ['Ġ',
  't',
  'o',
  'k',
  'e',
  'n',
  'i',
  'z',
  'a',
  't',
  'i',
  'o',
  'n'],
 'Ġsection': ['Ġ', 's', 'e', 'c', 't', 'i', 'o', 'n'],
 'Ġshows': ['Ġ', 's', 'h', 'o', 'w', 's'],
 'Ġseveral': ['Ġ', 's', 'e', 'v', 'e', 'r', 'a', 'l'],
 'Ġtokenizer': ['Ġ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r'],
 'Ġalgorithms': ['Ġ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's'],
 'Hopefully': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y'],
 ',': [','],
 'Ġyou': ['Ġ', 'y', 'o', 'u'],
 'Ġwill': ['Ġ', 'w', 'i', 'l', 'l'],
 'Ġbe': ['Ġ', 'b', 'e'],
 'Ġable': ['Ġ', 'a', 'b', 'l', 'e'],
 'Ġto': ['Ġ', 't', 'o'],
 'Ġunderstand': ['Ġ', 'u', 'n'

5. computes the frequency of each pair. We’ll need to use this at each step of the training

In [13]:
def compute_pair_freqs(splits):
    pair_freqs = {}
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            if pair not in pair_freqs:
               pair_freqs[pair] = 0
               pair_freqs[pair] += freq
            else: 
               pair_freqs[pair] += freq
    return pair_freqs

In [14]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7
('t', 'h'): 3


In [15]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 't') 7
